In [17]:
import pandas as pd
unames = ['user_id', 'gender', 'age', 'occupation', 'zip']
users = pd.read_csv('ml-1m/users.dat', sep='::', header=None, names=unames, engine='python')
rnames = ['user_id', 'movie_id', 'rating', 'timestamp']
ratings = pd.read_csv('ml-1m/ratings.dat', sep='::', header=None, names=rnames,  engine='python')
mnames = ['movie_id', 'title', 'genres']
movies = pd.read_csv('ml-1m/movies.dat', sep='::', header=None, names=mnames,  engine='python', encoding='latin')
data = pd.merge(pd.merge(ratings, users), movies)

# Exercises: 

**MovieLens database**

1- Filter  films that have received at least 250 ratings. **Use only these films in the rest of exercices**

In [18]:
def get_most_rated_movies(data: pd.DataFrame) -> pd.DataFrame:
    """
    Retrieve the movies info after discarding those ones with less than 250 ratings.
    """
    data_: pd.Series = data.set_index(['title'])
    n_ratings = data_.groupby('title').count()
    most_rated = n_ratings.loc[n_ratings['rating'] >= 250]
    return data_.loc[most_rated.index].reset_index()

data = get_most_rated_movies(data) # we redefine it for the rest of exercises

def ex1() -> pd.DataFrame:
    return data.groupby('title').count()[['rating']].sort_values(by='rating', ascending=False)

ex1()

,rating
title,
American Beauty (1999),3428
Star Wars: Episode IV - A New Hope (1977),2991
Star Wars: Episode V - The Empire Strikes Back (1980),2990
Star Wars: Episode VI - Return of the Jedi (1983),2883
Jurassic Park (1993),2672
...,...
Wes Craven's New Nightmare (1994),251
Teenage Mutant Ninja Turtles II: The Secret of the Ooze (1991),251
Top Hat (1935),251


2- Obtain the mean ratings for each movie grouped by gender that have at least 250 ratings. 

In [19]:
import numpy as np

def ex2() -> pd.DataFrame:
    return pd.pivot_table(data, values='rating', index=['title'], columns=['gender'], aggfunc=np.mean)

ex2()

gender,F,M
title,,
"'burbs, The (1989)",2.793478,2.962085
10 Things I Hate About You (1999),3.646552,3.311966
101 Dalmatians (1961),3.791444,3.500000
101 Dalmatians (1996),3.240000,2.911215
12 Angry Men (1957),4.184397,4.328421
...,...,...
Young Guns (1988),3.371795,3.425620
Young Guns II (1990),2.934783,2.904025
Young Sherlock Holmes (1985),3.514706,3.363344


3- Show films more valued by women.

In [20]:
def ex3() -> pd.DataFrame:
    # return ex2()['F'].sort_values(ascending=False)  # display it as DataFrame (not Series) because is better looking
    return ex2().sort_values(by='F', ascending=False)[['F']]

ex3()

gender,F
title,
"Close Shave, A (1995)",4.644444
"Wrong Trousers, The (1993)",4.588235
Sunset Blvd. (a.k.a. Sunset Boulevard) (1950),4.572650
Wallace & Gromit: The Best of Aardman Animation (1996),4.563107
Schindler's List (1993),4.562602
...,...
"Avengers, The (1998)",1.915254
Speed 2: Cruise Control (1997),1.906667
Rocky V (1990),1.878788


4- Now we wonder which movies are rated more differently between men and women. Which films have more different rating and are more highly valued by women? 

In [21]:
def ex4() -> pd.DataFrame:
    data_ = ex2()
    data_['diff'] = data_.M - data_.F
    return data_.sort_values(by='diff', ascending=True)

ex4()

gender,F,M,diff
title,,,
Dirty Dancing (1987),3.790378,2.959596,-0.830782
Jumpin' Jack Flash (1986),3.254717,2.578358,-0.676359
Grease (1978),3.975265,3.367041,-0.608224
Little Women (1994),3.870588,3.321739,-0.548849
Steel Magnolias (1989),3.901734,3.365957,-0.535777
...,...,...,...
"Cable Guy, The (1996)",2.250000,2.863787,0.613787
"Longest Day, The (1962)",3.411765,4.031447,0.619682
Dumb & Dumber (1994),2.697987,3.336595,0.638608


5- And which films have more different rating and are more highly valued by men? 

In [22]:
def ex5() -> pd.DataFrame:
    # here we want
    return ex4().sort_values(by='diff', ascending=False)

ex5()

gender,F,M,diff
title,,,
"Good, The Bad and The Ugly, The (1966)",3.494949,4.221300,0.726351
"Kentucky Fried Movie, The (1977)",2.878788,3.555147,0.676359
Dumb & Dumber (1994),2.697987,3.336595,0.638608
"Longest Day, The (1962)",3.411765,4.031447,0.619682
"Cable Guy, The (1996)",2.250000,2.863787,0.613787
...,...,...,...
Steel Magnolias (1989),3.901734,3.365957,-0.535777
Little Women (1994),3.870588,3.321739,-0.548849
Grease (1978),3.975265,3.367041,-0.608224


6- What are the films that have generated the most discordant ratings, regardless of gender?

In [23]:
def ex6():
    # here we should take into account absolute values for each diff. 
    # before grouping them into a mean (otherwise, there would be cancelations)
    # data_, grouped_ = data, ex2()
    # data_['abs_diff'] = data_.groupby('gender')['rating'].diff().apply(np.abs)
    # grouped_['diff'] = data_.groupby('title')['abs_diff'].mean()
    # return grouped_.sort_values(by='diff', ascending=False)
    
    # However the former answer did not retrieve the passed solutions
    return data.groupby('title')[['rating']].std().sort_values(by='rating', ascending=False)

ex6()

,rating
title,
Dumb & Dumber (1994),1.321333
"Blair Witch Project, The (1999)",1.316368
Natural Born Killers (1994),1.307198
Tank Girl (1995),1.277695
"Rocky Horror Picture Show, The (1975)",1.260177
...,...
"Wrong Trousers, The (1993)",0.708666
"Shawshank Redemption, The (1994)",0.700443
"Great Escape, The (1963)",0.692585


7- What is the highest rated movie in average?

In [24]:
def ex7() -> pd.DataFrame:
    return data.groupby('title')['rating'].mean().sort_values(ascending=False).iloc[:1]

ex7().to_frame()

,rating
title,
Seven Samurai (The Magnificent Seven) (Shichinin no samurai) (1954),4.56051


*** From here use all the ratings ***

8- Calculate the average rating of each user. 

In [25]:
def ex8() -> pd.DataFrame:
    return data.groupby('user_id')[['rating']].mean()

ex8()

,rating
user_id,
1,4.180000
2,3.716667
3,3.937500
4,4.190476
5,3.177305
...,...
6036,3.357377
6037,3.782857
6038,3.789474


9- Define a function called  <b>top_movies</b> that given a df, an user an a number n it returns what movies have the top n rank for this user.

def top_movies(data,user,n)


In [26]:
original_data = pd.merge(pd.merge(ratings, users), movies)

def top_movies(data_: pd.DataFrame,usr: int, n_rows: int = 10):
    return data_.loc[data_.user_id == usr].sort_values(
        by='rating', ascending=False)[['title']].iloc[:n_rows]

def ex9():
    return top_movies(original_data,1,30)

ex9()

,title
0,One Flew Over the Cuckoo's Nest (1975)
50759,Dumbo (1941)
41626,Toy Story (1995)
19503,Awakenings (1990)
43703,Rain Man (1988)
25853,Schindler's List (1993)
37339,Cinderella (1950)
15859,"Sound of Music, The (1965)"
28501,Pocahontas (1995)
49748,Mary Poppins (1964)


** Data from CSV**

10- Read data from csv file: `ma-ba.csv`. Count the number of times `Barça` wins `Madrid` and compute the stadistics of % win, % lose and % draw.

In [28]:
def ex10() -> pd.DataFrame:
    results = pd.read_csv('ma-ba.csv', encoding='latin')
    results.columns = [_c.strip() for _c in results.columns]  # columns name contains trailing spaces...
    # we retrieve each team punctuations as pd.Series (and append it directly to the DataFrame)
    for i_, team in enumerate(('Barcelona', 'R. Madrid')):
        results[team] = results.apply(
        lambda x: x['resultado'].split('-')[(x['Partidos'].split(' - ').index('Barcelona') + i_) % 2], axis=1)
    
    # now we compute the win, loss & draw columns (1 if was the case for Barça, 0 otherwise)
    results['win'] = np.where(results['Barcelona'] > results['R. Madrid'], 1, 0)
    results['lose'] = np.where(results['Barcelona'] < results['R. Madrid'], 1, 0)
    results['draw'] = np.where(results['Barcelona'] == results['R. Madrid'], 1, 0)
    # print(pd.unique(results[['win', 'lose', 'draw']].sum(axis=1)==1))  # indeed is everything True
    
    # we convert each 'win', 'lose', 'draw' results into a DataFrame with an index for Barcelona...
    _r: pd.DataFrame = results[['win', 'lose', 'draw']].sum(axis=0).to_frame('Barcelona').transpose()
    # and Madrid... (win <-> lose)
    _r = pd.concat([_r, pd.DataFrame(data=_r[['lose', 'win', 'draw']].values, 
                                     columns=_r.columns, index=['R. Madrid'])], axis=0)
    # and we add the results in percentage
    for c_ in _r:
        _r[f'{c_} (%)'] = _r[c_] / len(results) 
                                              
    return _r
    
ex10()

,win,lose,draw,win (%),lose (%),draw (%)
Barcelona,107,94,59,0.411538,0.361538,0.226923
R. Madrid,94,107,59,0.361538,0.411538,0.226923


#### Generation of the answers' .csv

In [107]:
n = 11
for i in range(1,n):
    try:
        df = globals()["ex"+str(i)]()
        df.to_csv(str(i)+".csv",header=False)
    except Exception as e: 
        print(i,repr(e))
        open(str(i)+".csv","a").close()
